In [4]:
import seqio
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
# Hard code for jupyter
file1 = "cds0.fa.gz" # true exons
file0 = "cdsf.fa.gz" # false 'randomly' generated sequences

In [7]:
# Convert true and false files into dataframe with labels, randomly shuffle
s1 = seqio.fasta2onehot(file1, '')
s0 = seqio.fasta2onehot(file0, '')
df1 = pd.DataFrame({"Sequence": s1, "Label": np.ones(len(s1))})
df0 = pd.DataFrame({"Sequence": s0, "Label": np.zeros(len(s0))})
df = df1.append(df0)
df = df.sample(frac=1, random_state=42)

In [8]:
# Create training and test features and labels
X = df["Sequence"]
Y = df["Label"]
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, stratify=Y)

In [18]:
# Compile Neural Network using list of layers
NN = keras.Sequential()
my_layers = [50 for i in range(3)]

# Input Layer
NN.add(keras.layers.Dense(my_layers[0], activation='sigmoid', input_shape=(204,)))

# Iterate through my_layers, use index to limit number
# for layer in my_layers:
NN.add(keras.layers.Dense(50))
NN.add(keras.layers.Dropout(0.2))
NN.add(keras.layers.Dense(3, kernel_regularizer='l2'))

# Output layer
NN.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile sequential model
opt = keras.optimizers.Adam(learning_rate=0.0001)
my_metrics = ['accuracy', 
              keras.metrics.Precision(name='Precision',thresholds=0.5),
              keras.metrics.Recall(name='Recall',thresholds=0.5)]
NN.compile(loss='binary_crossentropy', optimizer=opt, metrics=my_metrics)

In [19]:
e = 40
b = None
"""Not working because of a (32, 1) input shape ... 
 not sure where this is coming from as all x_train inputs are 204 long"""
result = NN.fit(x_train, y_train, epochs=e, batch_size=b)

Epoch 1/40


ValueError: in user code:

    /usr/local/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.9/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /usr/local/lib/python3.9/site-pack

In [ ]:
history = result.history

fig, axs = plt.subplots(1, 2)

axs[0].plot(history['accuracy'])
axs[0].plot(history['val_accuracy'])
axs[0].title('model accuracy')
axs[0].ylabel('accuracy')
axs[0].xlabel('epoch')
axs[0].legend(['train', 'test'], loc='upper left')

axs[1].plot(history.history['loss'])
axs[1].plot(history.history['val_loss'])
axs[1].title('model loss')
axs[1].ylabel('loss')
axs[1].xlabel('epoch')
axs[1].legend(['train', 'test'], loc='upper left')

plt.show()